In [ ]:
"""

Contents:
-data intialization from csv file of words &
 build data structure to house info moving forward
-write to .json formats
-read from existing .json formats
-get list of words to find
-scrape BBC
-scrape voiceTV
"""

In [1]:
import bs4, requests, sys, codecs, urllib.request, re
from bs4 import SoupStrainer
from bs4.element import Comment
import random
#import pythainlp
from pythainlp.tokenize import word_tokenize, sent_tokenize
import string
import json
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import pprint
import translators as ts

pp = pprint.PrettyPrinter()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 0)

path = "/Users/elyebliss/Desktop/Vocabulary/language_learning/vocab_dfs/"
source_file = "thai.json"

Using state Washington server backend.


In [2]:
##METHODS

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = bs4.BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)



user_agent = 'Mozilla/5.0 (Windows NT 6.3; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
headers={'User-Agent':user_agent,}
parser = 'html.parser'

In [3]:
def get_known(unknown_list):
    
    count_got = 0
    known_list = []
    for word in unknown_list:
        decision = str(input(word+"\nKnown =k"))
        if decision =='k':
            known_list.append(word)
            count_got +=1
            print("got "+str(count_got))
        elif decision=='q':
            break
        try:
            print(ts.google(word))
        except:
            print('cant find')
    return known_list

In [4]:
def filter_text(webpage,start=None,stop=None,\
                print_word_lvl=False,percent_threshold=None,\
               return_percent=False):


    try:
        request=urllib.request.Request(webpage,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read()
        contents = text_from_html(data)

        known_array = []
        unk_array = []
        contents_array = sent_tokenize(contents)

        if (start is not None) and (stop is not None):
            contents_array=contents_array[max(start,0):min(stop,len(contents_array))]

        disallowed_words = set()

        total_words = 0
        unknown_words = 0
        
        lines = []
        unknowns = []
        for line in contents_array:


            tokenized = word_tokenize(line)


            add_line = True
            line_total = 0
            line_unks = 0

            unk_str = ""
            for word in tokenized:
                
                total_words +=1
                line_total +=1

                if bool(re.search('[\u0E00-\u0E7F]+', word, flags=re.UNICODE)) and not ((word in vocab['white_listed']) or\
                                                            (word in vocab['black_listed'])):

                        unk_str += '"'+word+'"'+", "
                        if not percent_threshold:
                            add_line = False
                        disallowed_words.add(word)
                        unknown_words +=1
                        line_unks +=1

            if percent_threshold:
                if line_total>0:
                    if (1-(line_unks/line_total))<percent_threshold:
                        add_line = False
            if add_line:
                known_array.append(line)
                unk_array.append("...")
            else:
                known_array.append("...")
                unk_array.append(line)

            if len(unk_str)>0:
                unk_str = unk_str[0:len(unk_str)-2]

            unknowns.append(unk_str)

        if print_word_lvl and total_words>0:
            print("word-level % known = "+str((1-(unknown_words/total_words))*100))
        if return_percent and total_words>0:
            return (1-(unknown_words/total_words))*100
        return_pd = pd.DataFrame(list(zip(known_array,unk_array,unknowns)))
        return_pd.columns = ["knowns","unknowns","unk_words"]

        

        with open(path+'unknown_thai_list.txt',"w") as outfile:
            outfile.write(str(list(disallowed_words)))

        return return_pd

    except:
        pass

In [27]:
"""read from existing .json formats
uncovered, 
2164
2543
"""
with open(path+source_file, "r") as path_in:
    vocab = json.loads(path_in.read())
vocab['white_listed'] = set(vocab['white_listed'])
vocab['black_listed'] = set(vocab['black_listed'])
print(len(vocab['white_listed'])) 

#get 4k freq word info:
with open(path+'th_freq.csv','r') as infile:
    freq_df = pd.read_csv(infile)
uncovered = []
for word in freq_df.word:
    if word not in vocab['white_listed']:
        uncovered.append(word)
print(len(uncovered))



2164
2543


In [6]:
pages = []
titles = []

##BBC
print("Checking BBC Thai")
parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
request=urllib.request.Request('https://www.bbc.com/thai/topics/cjgn73g98rqt',None,headers)
resp = urllib.request.urlopen(request)
soup = bs4.BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))


for link in soup.find_all('a', href=True):
    if 'https://www.bbc.com/thai/thailand' in str(link['href']):
        pages.append(str(link['href']))
        request=urllib.request.Request(str(link['href']),None,headers) #The assembled request

        response = urllib.request.urlopen(request)
        data = response.read()
        contents = text_from_html(data)
        if bool(re.search("(?<=ยอดนิยม หน้าแรก ประเทศไทย ต่างประเทศ วิทยาศาสตร์ สุขภาพ โควิด-19 วิดีโอ ยอดนิยม).{30}",contents)):
            titles.append(re.findall("(?<=ยอดนิยม หน้าแรก ประเทศไทย ต่างประเทศ วิทยาศาสตร์ สุขภาพ โควิด-19 วิดีโอ ยอดนิยม).{30}",contents)[0].strip())

    elif '/thai/thailand' in str(link['href']):
        pages.append('https://www.bbc.com'+str(link['href']))

##VoiceTV
print("Checking VoiceTV")
parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
request=urllib.request.Request('https://www.voicetv.co.th/topic/%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B9%80%E0%B8%A1%E0%B8%B7%E0%B8%AD%E0%B8%87',None,headers)
resp = urllib.request.urlopen(request)
soup = bs4.BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))

 
for link in soup.find_all('a', href=True):
    str_page = 'https://www.voicetv.co.th'+str(link['href'])
    if '/read/' in str(link['href']) and str_page not in pages:
        pages.append(str_page)
        request=urllib.request.Request(str_page,None,headers) #The assembled request

        response = urllib.request.urlopen(request)
        data = response.read()
        contents = text_from_html(data)
        titles.append(re.findall("(?<=  :).{30}",contents)[0].strip())

Checking BBC Thai
Checking VoiceTV


In [7]:
#load prev corpus and add
known_corpus = set()
with open(path+"all_known_thai_lines.txt","r",encoding='utf-8') as infile:
    for line in infile.read().split('\n'):
        known_corpus.add(line)
len(known_corpus)

1144

In [8]:
#Find % of all sentences of news
knowns = []
all_lines = 0

known_percents = []

for webpage in pages:
    
    test_df = filter_text(webpage)
    all_lines += len(list(test_df.knowns))
    
    known_percents.append(filter_text(webpage,return_percent=True))
    
    for item in list(test_df.knowns[test_df.knowns != '...']):
        knowns.append(item)
percent = len(knowns)/float(all_lines)
print("percent all known= "+str(percent))


percent all known= 0.1476725521669342


In [9]:
#sorted(list(set(knowns)),key=len,reverse=True)
sorted(list(set(knowns).difference(known_corpus)),key=len,reverse=True)

['ดังนั้นสามารถพูดได้ทุกอย่าง อย่างเต็มที่ ยืนยัน การเมืองไม่ใช่อนาคตของตนเอง ',
 'ซึ่งหากมีการพูดคุยกันภายในฝั่งเจ้าหน้าที่ตำรวจคงจะไม่เกิดเหตุการณ์นั้นเลย ',
 'ไม่ใช่เมื่อถึงเวลาก็สามารถนำกำลังเข้ามาทำร้ายได้เลย ',
 'โดยผู้ชุมนุมขอว่าหากไปต่อไม่ได้ก็จะอยู่ตรงนี้ไปก่อน ',
 'ท่านมีจิตใจที่จะทำงานเพื่อสังคม แต่ไฟผมหมดแล้ว ',
 'Last update Nov 24, 2022 14:50 )   ‘สมหญิง’ ',
 'คนที่ 10 เมื่อ 7 ชั่วโมงที่แล้ว ',
 'เพราะหาก ท้องถิ่นจะเปิดสอบเอง ',
 'ส.ส.อำนาจฯ Nov 24, 2022 ( ',
 'และเห็นว่า บางตำแหน่ง ',
 'ท้องถิ่นไม่ควรทำเอง ',
 'ไม่ถึง 30 ล้านบาท ',
 'สังคมจะรับไม่ได้ ',
 'หน่วยงานของตนเอง ',
 'หากเกินกว่านี้ ',
 'ในวันดังกล่าว ',
 'ทำให้ดีที่สุด ',
 'ให้ท่านทำไป ',
 'ตรวจสอบได้ ',
 'ก็ดูแลด้วย ']

In [10]:
for item in list(set(knowns)):
    known_corpus.add(item)

with open(path+"all_known_thai_lines.txt","w",encoding='utf-8') as outfile:
    for line in list(set(known_corpus)):
        outfile.write(line+'\n')
len(known_corpus)

1164

In [11]:
websites = set()

with open(path+'viewed_websites_th.txt',"r") as infile:
    for line in infile.read().split('\n'):
        websites.add(line)
print(len(websites))

17


In [12]:
page_titles = list(zip(range(0,len(known_percents)),known_percents,titles,pages))
# (sorted(list(zip(range(0,len(known_percents)),known_percents)),key = lambda x: x[1],reverse=True))
page_titles = sorted(page_titles,key = lambda x: x[1],reverse=True)
page_titles = [item for item in page_titles if not (item[3] in websites)]
page_titles

[(2,
  87.88426763110307,
  "'กมธ.พัฒนาการเมืองฯ' เชิญ 'รา",
  'https://www.voicetv.co.th/read/9OrSFO8wi'),
 (5,
  87.40416210295729,
  '‘วิษณุ’แย้มประตูยุบสภาเปิดอยู',
  'https://www.voicetv.co.th/read/WRfVuH5Ed'),
 (3,
  86.46288209606988,
  '‘จิราพร’ ให้ ‘ประยุทธ์’ สอบตก',
  'https://www.voicetv.co.th/read/ZKWwEQwfG'),
 (4,
  86.2240663900415,
  '‘ประยุทธ์’ อวยพรปีใหม่สื่อทำเ',
  'https://www.voicetv.co.th/read/YSRkY2I1d'),
 (0,
  85.68738229755179,
  '‘สมหญิง’พร้อมสู้คดีฟุตซอล ยัน',
  'https://www.bbc.com/thai/thailand-63662204'),
 (1,
  81.8326439707286,
  "'อนุพงษ์' พบผู้บริหาร อบต. ทั",
  'https://www.bbc.com/thai/thailand-63566397'),
 (6,
  80.27923211169285,
  "รีวิว 'Collective': นักข่าวไม",
  'https://www.voicetv.co.th/read/bn95YMr0a')]

In [13]:
webpage = pages[2]
if webpage in websites:
    print("already scanned! choose another!")
websites.add(webpage)

In [14]:

with open(path+'viewed_websites_th.txt',"w") as outfile:
    for line in websites:
        outfile.write(line+'\n')      


In [15]:
#display() #percent_threshold=0.95
art=filter_text(webpage,print_word_lvl=True)

word-level % known = 87.88426763110307


In [16]:
with open(path+'unknown_thai_list.txt',"r") as input_file:
    new_words = input_file.read()
    new_words = re.sub("[\n\'\[\]]","",new_words)
    new_words = new_words.split(',')
    new_words = [line.strip() for line in new_words] #update regex
#new_words
print(len(new_words))
#known_manual = get_known(new_words)

50


In [17]:
import mitosheet
unk_df = pd.DataFrame(new_words)
unk_df.columns = ['word']
unk_df['status'] = pd.Series(['' for word in new_words])

In [18]:
mitosheet.sheet(unk_df)

MitoWidget(analysis_data_json='{"analysisName": "id-bzlqgvmqzj", "analysisToReplay": null, "code": [], "stepSu…

In [19]:
add_ints = [20, 28, 33, 37, 42, 44, 45, 48, 49]

In [20]:
for i in add_ints:
    # Set a cell value in status
    unk_df.at[i, 'status'] = 'k'

In [21]:
translations = []
for i in range(0,len(unk_df)):
    try:
        translations.append(ts.google(unk_df.word.iloc[i]))
    except:
        translations.append('')
unk_df['translations']=translations


In [22]:
mitosheet.sheet(unk_df, analysis_to_replay="id-cofchfkpsa")

MitoWidget(analysis_data_json='{"analysisName": "id-cofchfkpsa", "analysisToReplay": null, "code": [], "stepSu…

In [23]:
add_words = list(unk_df.word[unk_df['status']=='k'])
print(len(add_words))
for word in add_words:
    vocab['white_listed'].add(word)

9


In [56]:
#manually remove from white
remove_from_white = []

for word in remove_from_white:
    if word in vocab['white_listed']:
        vocab['white_listed'].remove(word)

In [55]:
#manually add to white
add_to_white = []

for word in add_to_white:
    vocab['white_listed'].add(word)

In [24]:
#write to .json formats
df = vocab
df['white_listed'] = list(df['white_listed'])
df['black_listed'] = list(df['black_listed'])
with open(path+source_file, "w") as outfile:
    json.dump(df,outfile)
print(len(df['white_listed']))

2164


In [25]:
#read from existing .json formats
with open(path+source_file, "r") as path_in:
    vocab = json.loads(path_in.read())
vocab['white_listed'] = set(vocab['white_listed'])
vocab['black_listed'] = set(vocab['black_listed'])
print(len(vocab['white_listed']))

2164


In [26]:
print(webpage)
filter_text(webpage,print_word_lvl=True)

https://www.voicetv.co.th/read/9OrSFO8wi
word-level % known = 90.23508137432188


,knowns,unknowns,unk_words
0,POLITICS ECONOMICS WORLD ENTERTAINMENT WELL-BEING LOCAL NEWS BLOG VOICE PLAZA TV PROGRAMS LIVE POLITICS ECONOMICS WORLD ENTERTAINMENT WELL-BEING LOCAL NEWS BLOG VOICE PLAZA TV PROGRAMS ABOUT FAQ CONTACT TERM OF USE SCHEDULE ไม่พบผลการค้นหา,...,
1,คุณกำลังอ่าน : ‘สมหญิง’พร้อมสู้คดีฟุตซอล,...,
2,...,ยันไม่กระทบลงชิง,"""ยัน"", ""ชิง"""
3,ส.ส.อำนาจฯ Share Tweet Share การเมือง ‘สมหญิง’พร้อมสู้คดีฟุตซอล,...,
4,...,ยันไม่กระทบลงชิง,"""ยัน"", ""ชิง"""
...,...,...,...
15,เมื่อมีผู้มาร้องเรียนจึงต้องมีการตรวจสอบเพื่อความถูกต้อง เพื่อความสบายใจของทุกฝ่าย,...,
16,...,“ในการลงสมัครรับเลือกตั้งนั้น ดิฉันขอยืนยันว่าลงสมัครรับเลือกตั้งได้,"""ลงสมัครรับเลือกตั้ง"", ""ลงสมัครรับเลือกตั้ง"""
17,...,ขอให้พี่น้องประชาชนให้กำลังใจซึ่งอาจมีบุคคลที่จ้องจะทำลายชื่อเสียง,"""จ้อง"""
18,...,มั่นใจว่ากรณีที่เกิดขึ้นมาไม่มีผลต่อการเป็นผู้สมัครสมาชิกสภาผู้แทนราษฎร จังหวัดอำนาจเจริญแต่อย่างใด พร้อมที่จะลงสมัคร ส.ส.เพื่อเป็นตัวเลือกให้กับประชาชนในจังหวัด พร้อมสู้ทุกกติกาและมั่นใจว่าจากผลงานที่ตนทำมาและเข้าถึงพี่น้องประชาชนมาตลอด,"""มีผลต่อ"", ""สมาชิกสภาผู้แทนราษฎร"", ""อำนาจเจริญ"", ""ลงสมัคร"", ""กติกา"""


##END

In [147]:
#Upload from download
with open(path+'known_thai_list.txt',"r") as input_file:
    new_words = input_file.read().split(',')
    print("daily catch in word count:")
    print(new_words)
for line in new_words:   
    vocab['white_listed'].add(line.replace("'","").strip())


daily catch in word count:
['เชิญ', 'ยิ้ม', 'ยินดี', 'รอย', 'เอกสาร', 'สนิท', 'กระดาษ', 'คอ', 'โรงพยาบาล', 'หิน', 'จด', 'ปืน', 'โต๊ะ', 'เสื้อผ้า', 'เลือด', 'สิทธิ์', 'นักเขียน', 'สนุกสนาน', 'หัวเราะ', 'หู', 'ยิง', 'สวยงาม', 'ล่า', 'แขก', 'ดอกไม้', 'ที่นั่ง', 'เสือ', 'ปกป้อง', 'รับประทาน', 'ไอ', 'สนุก', 'ลูกชาย', 'สหรัฐ', 'นิ้ว', 'ไอ้', 'รถไฟ', 'เล็กน้อย', 'โปรด', 'คะแนน', 'กรุณา', 'โรงแรม', 'จดหมาย', 'ขับ', 'ดื่ม', 'หอม', 'ระมัดระวัง', 'ฤดู', 'ฟัน', 'เท้า', 'หล่อน', 'วางแผน', 'ลับ', 'ลอย', 'ยกเว้น', 'ลูกค้า', 'ริม', 'ม้า', 'คอมพิวเตอร์']


In [154]:

wyw_text = 'สวัสดีครับน้อง'
print(ts.google(wyw_text))
#print(ts._google.language_map)
#print(ts.google(wyw_text))

Hello Nong


In [227]:
#Get after when adding in new words
print("today's catch % of corpus:")
str(100*(percent-prev_percent))+"%"

today's catch % of corpus:


'1.200344766395417%'

In [ ]:
#output newly knowns
with open(path+"new_known_thai_lines.txt","w") as outfile:
    for line in list(set(knowns).difference(prev_knowns)):
        outfile.write(line+'\n')


In [105]:
print("Sanity check. Current vocab size:")
len(vocab['white_listed'])

Sanity check. Current vocab size:


1462

In [117]:
"""
to do:

"""

In [7]:
##INPUT-OUTPUT
#vocab list:
with open("/Users/elyebliss/Desktop/Vocabulary/vocab_dfs/thai_whitelisted.csv","r") as infile:
    whitelisted_lemmas = infile.read()


##VARIABLES
vocab_all = set()


for line in whitelisted_lemmas.split('\n'):
    if len(line) > 0:
        
        vocab = line.strip()
        vocab_all.add(vocab)
        
len(whitelisted_lemmas.split('\n'))            
#pp.pprint(vocab_all)

999

In [8]:
vocab = {}
vocab['white_listed'] = list(vocab_all)
vocab['black_listed'] = []